<a href="https://colab.research.google.com/github/DarthCoder501/ML-AI-Projects/blob/main/Codebase%20RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Necessary Libraries

In [ ]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

# Clone a GitHub Repo locally

In [ ]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = repo_url.split("/")[-1]  # Extract repository name from URL
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)

In [ ]:
path = clone_repository("https://github.com/CoderAgent/SecureAgent")

In [ ]:
print(path)

In [ ]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [ ]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Get relative path from repo root
        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
        for root, _, files in os.walk(repo_path):
            # Skip if current directory is in ignored directories
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            # Process each file in current directory
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

In [ ]:
file_content = get_main_files_content(path)

In [ ]:
file_content

# Embeddings

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [ ]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

In [ ]:
embeddings

# Setting up Pinecone



In [ ]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [ ]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

In [ ]:
documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)

# Perform RAG


In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [ ]:
query = "How are typescript files parsed?"

In [ ]:
raw_query_embedding = get_huggingface_embeddings(query)

raw_query_embedding

In [ ]:
# Feel free to change the "top_k" parameter to be a higher or lower number
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

In [ ]:
top_matches

In [ ]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
contexts

In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(augmented_query)

In [ ]:
system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
"""

llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [ ]:
response

# Putting it all together

In [ ]:
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [ ]:
response = perform_rag("How is the javascript parser used?")

print(response)

# Buidling Web App

In [ ]:
!pip install streamlit pyngrok python-dotenv

In [ ]:
!pip install groq


In [ ]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata
import os

In [ ]:
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

ngrok.set_auth_token(ngrok_token)

In [ ]:
def run_streamlit():
  os.system('streamlit run /content/app.py --server.port 8501')

In [ ]:
%%writefile app.py
import streamlit as st
from rag import perform_rag  # Import the perform_rag function from your rag.py file

# Set the Streamlit app configuration
st.set_page_config(page_title="Codebase Chatbot", layout="wide")

# Title and Description
st.title("Codebase Chatbot")
st.write(
    """
    Welcome to your codebase chatbot! Engage in a conversation about your codebase,
    and receive detailed answers based on the embedded context.
    """
)

# Initialize session state for storing chat history
if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "Hi! I'm here to answer your questions about the codebase. How can I assist you today?"}]  # List to hold chat messages

# Display previous messages in the chat
for message in st.session_state.messages:
    with st.chat_message(message["role"]):  # `role` can be "user" or "assistant"
        st.markdown(message["content"])  # Render the message content

# Input box for user query
if user_input := st.chat_input("Ask your question about the codebase..."):
    # Add user message to session state
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)  # Display user's message

    # Process the query with the RAG function
    with st.chat_message("assistant"):
        with st.spinner("Processing your query..."):
            try:
                # Call perform_rag to generate a response
                response = perform_rag(user_input)
                st.markdown(response)  # Display assistant's response
                # Add assistant's response to session state
                st.session_state.messages.append({"role": "assistant", "content": response})
            except Exception as e:
                error_message = f"An error occurred: {e}"
                st.error(error_message)
                st.session_state.messages.append({"role": "assistant", "content": error_message})

In [ ]:
%%writefile rag.py
import requests
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Set up API keys and configuration
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = 'us-east-1'
PINECONE_INDEX_NAME = 'codebase-rag'
PINECONE_NAMESPACE = 'https://github.com/CoderAgent/SecureAgent'
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_MODEL = "llama-3.1-70b-versatile"

def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

# Initialize Pinecone
pc = Pinecone(
    api_key= PINECONE_API_KEY,
    spec=ServerlessSpec(cloud="aws", region=PINECONE_ENVIRONMENT)
)

pinecone_index = pc.Index(PINECONE_INDEX_NAME)
index_description = pc.describe_index(PINECONE_INDEX_NAME)
print(index_description)
def perform_rag(query):
    """
    Perform Retrieval-Augmented Generation (RAG) to answer a query.

    Args:
        query (str): User query.

    Returns:
        str: LLM-generated response.
    """
    # Step 1: Generate embeddings for the query
    raw_query_embedding = get_huggingface_embeddings(query)
    raw_query_embedding_list = raw_query_embedding.tolist()

    # Step 2: Query Pinecone for relevant contexts
    top_matches = pinecone_index.query(
        vector=raw_query_embedding_list,
        top_k=5,
        include_metadata=True,
        namespace=PINECONE_NAMESPACE
    )

    # Extract contexts from matches
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    # Step 3: Construct an augmented query
    augmented_query = (
        "<CONTEXT>\n"
        + "\n\n-------\n\n".join(contexts[:10])  # Use top 10 results
        + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query
    )

    # Step 4: Define the system prompt
    system_prompt = (
        "You are a Senior Software Engineer specializing in Python and JavaScript. "
        "You are an expert in the Django framework.\n\n"
        "Answer the following question about the codebase using the context provided. "
        "Always explain your reasoning step by step."
    )

    # Step 5: Query the Llama 3.1 API
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
         "model": GROQ_MODEL,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ],
        "max_tokens": 1024,
        "temperature": 0.7
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)
    # Debugging Groq response

    if response.status_code == 200:
        response_json = response.json()
        try:
            return response_json["choices"][0]["message"]["content"]
        except (KeyError, IndexError):
            return "No response text found in the Groq API response."
    else:
        raise ValueError(f"Error from Llama API: {response.status_code} - {response.text}")

In [ ]:
thread = Thread(target=run_streamlit)
thread.start()

In [ ]:
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)

print("Public URL: ", public_url)

In [ ]:
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
  print(f"Closing tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")
  ngrok.disconnect(tunnel.public_url)

In [ ]:
%%writefile .env

GROQ_API_KEY = <Insert GROQ API key here>
NGROK_AUTH_TOKEN = <Insert NGROK authentication token here>
PINECONE_API_KEY = <Insert Pinecone API key here>